### Imports

In [1]:
from tqdm import tqdm
from utils.utils import *

import networkx as nx
import numpy as np
import pandas as pd
import os
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 180 µs (started: 2023-05-29 13:29:42 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

/home/ealvarez/Project/GNN_Filter
Group:		 ['WT', 5]
time: 2.6 ms (started: 2023-05-29 13:29:42 -05:00)


### Load dataset

In [3]:
# Load dataset Groups
df1 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int1"), delimiter="|")
df2 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int2"), delimiter="|")
df3 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int3"), delimiter="|")
df4 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int4"), delimiter="|")
# df5_ = pd.read_csv("{}/inputs/Edwin_proyecto2/{}.csv".format(dir, "int5"), delimiter="|")

""" df1 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities1"), delimiter="|")
df2 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities2"), delimiter="|")
df3 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities3"), delimiter="|")
df4 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities4"), delimiter="|")
df5_ = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities5"), delimiter="|") """

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
# print(df5_.shape)

(6245, 102)
(6245, 72)
(6245, 97)
(6245, 137)
time: 209 ms (started: 2023-05-29 13:29:42 -05:00)


In [4]:
# concat
# df_join_raw = pd.concat([df1.iloc[:,1:], df2.iloc[:, 2:], df3.iloc[:, 2:], df4.iloc[:, 2:], df5.iloc[:, 2:]], axis=1)
df_join_raw = pd.concat([df1.iloc[:, 1:], df2.iloc[:, 2:], df3.iloc[:, 2:], df4.iloc[:, 2:]], axis=1)
df_join_raw.set_index("ionMz", inplace=True)

print(df_join_raw.shape)
df_join_raw

(6245, 400)


,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1,47,508,360,675,232,59,345,346,504,...,45,817,32,307,410,716,228,187,361,952
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,83,174,366,1134,693,464,235,479,443,691,...,225,349,185,429,186,278,364,368,115,887
59.0453,1,1,51,642,493,143,11,227,160,154,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433


time: 29.4 ms (started: 2023-05-29 13:29:42 -05:00)


In [5]:
print(list(df_join_raw.index))

[59.0049, 59.0137, 59.0291, 59.037, 59.0453, 60.0171, 60.9871, 61.9884, 71.0139, 74.0249, 78.9592, 78.9655, 85.0296, 87.0086, 88.0405, 89.0245, 94.9806, 96.9602, 96.9688, 96.9761, 96.9954, 97.9758, 98.9721, 98.9911, 99.9716, 100.9337, 100.9679, 101.002, 101.0243, 102.0562, 102.9758, 107.0363, 109.006, 109.0407, 109.0514, 110.0721, 111.0086, 111.02, 111.0285, 111.0409, 112.012, 112.0232, 112.0518, 112.9857, 113.0246, 113.0355, 113.0606, 114.0194, 114.0276, 114.056, 115.0038, 115.0401, 115.0595, 115.0764, 115.9206, 116.0356, 116.0507, 116.0718, 116.092, 116.9287, 117.0195, 117.0546, 117.0753, 118.0222, 118.051, 118.0721, 118.0812, 118.9242, 118.9421, 118.957, 118.9807, 119.0351, 119.054, 119.9244, 120.0553, 120.931, 120.9537, 120.9764, 120.9957, 121.03, 121.0383, 121.997, 122.0247, 122.9697, 122.991, 123.0007, 123.0091, 123.0188, 123.0266, 123.036, 123.0462, 123.0597, 123.9693, 123.9906, 124.0003, 124.0184, 124.0403, 124.9664, 124.9894, 124.9971, 125.0076, 125.0241, 126.03, 127.0513, 127

In [10]:
# get WT group
df_wt = df_join_raw.filter(like="WT", axis=1)

print(df_wt.shape)
df_wt

(6245, 200)


,0071 / WT 2.2,0072 / WT 2.2,0073 / WT 2.2,0074 / WT 2.2,0075 / WT 2.2,0076 / WT 2.2,0077 / WT 2.2,0078 / WT 2.2,0079 / WT 2.2,0080 / WT 2.2,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,528,1,378,438,530,250,662,431,305,334,...,45,817,32,307,410,716,228,187,361,952
59.0137,56523,24625,78586,76559,67449,74919,88152,99718,86001,93962,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2290,927,3149,3038,2774,3419,3911,3820,3374,3746,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,320,195,1137,642,588,799,602,606,768,372,...,225,349,185,429,186,278,364,368,115,887
59.0453,196,59,555,405,234,413,46,148,95,25,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2050,1415,3851,2285,2038,3089,2737,2788,1819,2600,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,2681,1984,3570,2169,2479,2668,3033,3271,2264,2411,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,1516,1307,2299,2363,2877,1436,1771,2326,1629,2740,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433


time: 18.7 ms (started: 2023-05-30 16:00:02 -05:00)


In [88]:
# get ZW group
df_zw = df_join_raw.filter(like="zwf1^")

print(df_zw.shape)
df_zw

(6245, 120)


,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0861 / zwf1^ 1.4,0862 / zwf1^ 1.4,0863 / zwf1^ 1.4,0864 / zwf1^ 1.4,0865 / zwf1^ 1.4,0866 / zwf1^ 1.4,0867 / zwf1^ 1.4,0868 / zwf1^ 1.4,0869 / zwf1^ 1.4,0870 / zwf1^ 1.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1,47,508,360,675,232,59,345,346,504,...,344,790,851,372,899,752,54,254,145,1236
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,134377,133636,129520,122223,133150,129628,127563,116371,114443,118647
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,5207,5676,5416,5091,5594,5638,5234,4746,4308,5475
59.0370,83,174,366,1134,693,464,235,479,443,691,...,939,946,1195,700,1071,1302,993,857,412,1348
59.0453,1,1,51,642,493,143,11,227,160,154,...,225,168,398,84,76,516,127,351,1,753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1647,3001,2639,2850,2716,2415,2060,2886,2558,2891
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,2002,3333,2695,2343,2433,2471,2424,2671,2323,2457
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,2782,2553,2499,1393,2354,2002,2307,2151,2031,1425


time: 27 ms (started: 2023-05-24 12:02:03 -05:00)


In [89]:
# get pc group
df_pc = df_join_raw.filter(like="pck1^")

print(df_pc.shape)
df_pc

(6245, 80)


,0011 / pck1^ 2.4,0012 / pck1^ 2.4,0013 / pck1^ 2.4,0014 / pck1^ 2.4,0015 / pck1^ 2.4,0016 / pck1^ 2.4,0017 / pck1^ 2.4,0018 / pck1^ 2.4,0019 / pck1^ 2.4,0020 / pck1^ 2.4,...,0881 / pck1^ 2.3,0882 / pck1^ 2.3,0883 / pck1^ 2.3,0884 / pck1^ 2.3,0885 / pck1^ 2.3,0886 / pck1^ 2.3,0887 / pck1^ 2.3,0888 / pck1^ 2.3,0889 / pck1^ 2.3,0890 / pck1^ 2.3
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,901,81,377,488,883,426,421,689,383,222,...,857,655,431,335,200,670,256,288,315,427
59.0137,33968,22565,36285,30275,34858,28558,36375,31684,25583,24293,...,59883,65860,65463,64715,52758,56120,60780,64502,62149,66849
59.0291,1947,945,2010,1776,1806,1549,2015,1494,1399,1029,...,2528,3118,2863,2896,2422,2483,2878,3035,2456,2964
59.0370,644,139,752,607,694,685,681,404,619,117,...,522,911,995,655,740,733,652,693,697,724
59.0453,139,78,352,414,373,388,376,353,345,134,...,181,459,429,155,321,257,342,135,315,415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,1169,923,815,1219,638,1161,1461,1710,1224,947,...,1580,1340,1940,1714,1266,735,828,1709,978,1051
996.7096,1122,1767,1914,1748,1575,1599,1738,1701,2425,1651,...,1330,817,1456,1113,816,520,161,1229,596,447
997.5542,1135,1849,1313,819,1883,1296,964,1236,1117,711,...,1111,1401,467,2230,1664,1296,391,587,883,762


time: 23.1 ms (started: 2023-05-24 12:02:03 -05:00)


In [17]:
# Logarithm

if group[0] == "WT": # Unable this line when it's run by group
  df_join_raw = df_wt.copy()
elif group[0] == "zwf1^":
  df_join_raw = df_zw.copy()
elif group[0] == "pck1^":
  df_join_raw = df_pc.copy()

df_raw_log = df_join_raw.copy()
for column in df_join_raw.columns:
  df_raw_log[column] = np.log10(df_join_raw[column])

df_raw_log

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,0071 / WT 2.2,0072 / WT 2.2,0073 / WT 2.2,0074 / WT 2.2,0075 / WT 2.2,0076 / WT 2.2,0077 / WT 2.2,0078 / WT 2.2,0079 / WT 2.2,0080 / WT 2.2,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,2.722634,0.000000,2.577492,2.641474,2.724276,2.397940,2.820858,2.634477,2.484300,2.523746,...,1.653213,2.912222,1.505150,2.487138,2.612784,2.854913,2.357935,2.271842,2.557507,2.978637
59.0137,4.752225,4.391376,4.895345,4.883996,4.828976,4.874592,4.945232,4.998774,4.934504,4.972952,...,4.660363,4.598539,4.673988,4.587677,4.734960,4.796644,4.674705,4.698335,4.639805,4.685840
59.0291,3.359835,2.967080,3.498173,3.482588,3.443106,3.533899,3.592288,3.582063,3.528145,3.573568,...,3.267641,3.234770,3.262451,3.220108,3.302331,3.404663,3.322839,3.357172,3.215638,3.401745
59.0370,2.505150,2.290035,3.055760,2.807535,2.769377,2.902547,2.779596,2.782473,2.885361,2.570543,...,2.352183,2.542825,2.267172,2.632457,2.269513,2.444045,2.561101,2.565848,2.060698,2.947924
59.0453,2.292256,1.770852,2.744293,2.607455,2.369216,2.615950,1.662758,2.170262,1.977724,1.397940,...,1.778151,2.466868,0.000000,2.526339,1.812913,2.152288,0.000000,2.260071,1.000000,2.835056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.311754,3.150756,3.585574,3.358886,3.309204,3.489818,3.437275,3.445293,3.259833,3.414973,...,3.270912,3.263399,3.222456,2.828660,3.403635,3.007321,3.474653,3.258637,3.334454,3.278525
996.7096,3.428297,3.297542,3.552668,3.336260,3.394277,3.426186,3.481872,3.514681,3.354876,3.382197,...,3.242541,3.151982,3.064458,2.793092,3.245019,3.168497,3.295127,3.009876,3.247728,3.211921
997.5542,3.180699,3.116276,3.361539,3.373464,3.458940,3.157154,3.248219,3.366610,3.211921,3.437751,...,3.067443,2.918030,3.065206,3.042576,3.352761,3.151982,3.362859,3.188084,2.917506,3.156246


time: 83.3 ms (started: 2023-05-30 16:02:32 -05:00)


In [23]:
print(list(df_raw_log.iloc[:, 3].values[:2540]))

[2.6414741105040997, 4.88399625212879, 3.4825877695267677, 2.807535028068853, 2.6074550232146687, 3.3919930722597127, 2.599883072073688, 3.3236645356081, 3.104487111312395, 3.3981136917305026, 2.91539983521227, 2.7528164311882715, 3.1945143418824675, 3.2234959409623944, 3.0354297381845483, 3.4342494523964757, 3.4265112613645754, 3.74499667403856, 3.502290527914773, 3.6085260335771943, 2.841359470454855, 2.7902851640332416, 3.343802333161655, 2.330413773349191, 2.56702636615906, 2.9656719712201065, 3.2068258760318495, 3.081707270097349, 3.094471128641645, 2.683947130751512, 3.360593413565249, 3.091315159697223, 3.1126050015345745, 3.413467412985825, 3.0718820073061255, 2.90687353472207, 4.007833092701319, 4.277609214304091, 3.7545776560447304, 3.0773679052841567, 2.8549130223078554, 3.0472748673841794, 3.4896772916636984, 3.2030328870147105, 3.681060243631812, 3.1818435879447726, 2.9380190974762104, 2.9609461957338317, 3.0354297381845483, 3.353916230920363, 3.05307844348342, 3.139249217

In [91]:
subgroups = [item.split("{} ".format(group[0]))[1].split(".")[0] for item in list(df_raw_log.columns)]
subgroups = np.unique(subgroups)
subgroups

array(['1', '2', '3', '4', '5'], dtype='<U1')

time: 4.55 ms (started: 2023-05-24 12:02:04 -05:00)


In [92]:
# Split graph

list_raw = []

for item in subgroups:
  list_raw.append(df_raw_log.filter(like="{} {}.".format(group[0], item)))

# list_raw = [df_raw_log] # only for create one graph
print(len(list_raw))
list_raw[0]

5


,0091 / WT 1.3,0092 / WT 1.3,0093 / WT 1.3,0094 / WT 1.3,0095 / WT 1.3,0096 / WT 1.3,0097 / WT 1.3,0098 / WT 1.3,0099 / WT 1.3,0100 / WT 1.3,...,0821 / WT 1.4,0822 / WT 1.4,0823 / WT 1.4,0824 / WT 1.4,0825 / WT 1.4,0826 / WT 1.4,0827 / WT 1.4,0828 / WT 1.4,0829 / WT 1.4,0830 / WT 1.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,2.515874,2.453318,2.832509,2.687529,2.515874,1.973128,2.873902,2.670246,2.795185,2.675778,...,2.686636,2.658011,2.689309,2.902003,2.872739,2.378398,2.842609,2.836957,3.044148,2.672098
59.0137,5.101637,5.073656,5.191384,5.053144,5.127630,5.014327,5.236834,5.009944,5.198135,5.278424,...,4.904088,4.937468,4.943069,4.867391,4.962649,4.961786,4.989499,4.914708,4.984023,4.986274
59.0291,3.683497,3.647872,3.772762,3.684486,3.743196,3.626853,3.847819,3.593618,3.759290,3.860038,...,3.530072,3.522053,3.543323,3.539076,3.593618,3.594614,3.639088,3.568671,3.661623,3.615529
59.0370,2.841985,2.990783,3.060320,3.089198,3.143951,2.880242,3.228657,2.829947,2.930949,3.070776,...,2.990339,3.022428,2.748963,2.927883,2.843233,2.973128,2.937518,3.072985,2.995196,2.896526
59.0453,0.000000,2.475671,2.632457,2.597695,2.862131,2.468347,2.568202,2.553883,2.403121,2.113943,...,2.499687,2.424882,0.000000,2.588832,2.459392,2.665581,2.596597,2.874482,2.693727,2.397940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.431203,3.400538,3.375481,3.477266,3.579669,3.106191,3.448397,3.398461,3.493179,3.498724,...,3.274620,3.240549,3.348500,2.993436,3.167022,3.196729,3.121560,3.302980,3.147985,3.387034
996.7096,2.877371,3.203305,3.030600,3.000000,3.068557,3.232996,2.970812,3.075547,3.157457,3.283075,...,2.978181,3.010724,3.106191,2.926342,2.813581,2.968016,2.629410,2.843855,3.015360,3.181272
997.5542,3.253822,3.159868,3.400538,3.302547,3.332842,3.306639,3.492201,3.340841,3.473633,3.412461,...,2.882525,3.237795,2.848189,2.903090,3.129045,2.842609,3.232742,3.332438,3.104828,2.862728


time: 37.4 ms (started: 2023-05-24 12:02:04 -05:00)


In [93]:
for k in range(len(subgroups)):
    print(len(list_raw[k].columns), list(list_raw[k].columns))

40 ['0091 / WT 1.3', '0092 / WT 1.3', '0093 / WT 1.3', '0094 / WT 1.3', '0095 / WT 1.3', '0096 / WT 1.3', '0097 / WT 1.3', '0098 / WT 1.3', '0099 / WT 1.3', '0100 / WT 1.3', '0751 / WT 1.1', '0752 / WT 1.1', '0753 / WT 1.1', '0754 / WT 1.1', '0755 / WT 1.1', '0756 / WT 1.1', '0757 / WT 1.1', '0758 / WT 1.1', '0759 / WT 1.1', '0760 / WT 1.1', '0811 / WT 1.2', '0812 / WT 1.2', '0813 / WT 1.2', '0814 / WT 1.2', '0815 / WT 1.2', '0816 / WT 1.2', '0817 / WT 1.2', '0818 / WT 1.2', '0819 / WT 1.2', '0820 / WT 1.2', '0821 / WT 1.4', '0822 / WT 1.4', '0823 / WT 1.4', '0824 / WT 1.4', '0825 / WT 1.4', '0826 / WT 1.4', '0827 / WT 1.4', '0828 / WT 1.4', '0829 / WT 1.4', '0830 / WT 1.4']
40 ['0071 / WT 2.2', '0072 / WT 2.2', '0073 / WT 2.2', '0074 / WT 2.2', '0075 / WT 2.2', '0076 / WT 2.2', '0077 / WT 2.2', '0078 / WT 2.2', '0079 / WT 2.2', '0080 / WT 2.2', '0271 / WT 2.3', '0272 / WT 2.3', '0273 / WT 2.3', '0274 / WT 2.3', '0275 / WT 2.3', '0276 / WT 2.3', '0277 / WT 2.3', '0278 / WT 2.3', '0279 

In [98]:
list_raw[4]

,0361 / WT 5.1,0362 / WT 5.1,0363 / WT 5.1,0364 / WT 5.1,0365 / WT 5.1,0366 / WT 5.1,0367 / WT 5.1,0368 / WT 5.1,0369 / WT 5.1,0370 / WT 5.1,...,0921 / WT 5.4,0922 / WT 5.4,0923 / WT 5.4,0924 / WT 5.4,0925 / WT 5.4,0926 / WT 5.4,0927 / WT 5.4,0928 / WT 5.4,0929 / WT 5.4,0930 / WT 5.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,2.830589,2.792392,0.698970,2.629410,3.200850,2.542825,2.348305,2.562293,2.267172,2.456366,...,2.802089,3.054613,1.113943,2.655138,2.492760,2.267172,2.763428,1.863323,2.361728,2.149219
59.0137,4.811179,4.451602,4.763885,4.835526,4.715945,4.759894,4.674466,4.822920,4.784874,4.779676,...,4.675879,4.755280,4.687681,4.671071,4.633378,4.749458,4.704485,4.698370,4.683947,4.712448
59.0291,3.350442,3.217747,3.263399,3.432649,3.455606,3.393751,3.318898,3.483302,3.409426,3.359456,...,3.373831,3.409764,3.314710,3.294687,3.278754,3.404149,3.325516,3.282622,3.417804,3.319314
59.0370,2.752816,2.868644,2.575188,2.650308,2.905796,2.698101,2.517196,2.691965,2.773055,2.409933,...,2.839478,2.905796,2.620136,2.814248,2.836957,2.610660,2.742725,2.378398,2.945961,2.668386
59.0453,2.820858,2.781755,2.201397,1.812913,2.768638,1.672098,2.397940,1.755875,2.517196,2.049218,...,2.495544,2.622214,1.770852,2.745075,2.730782,2.356026,2.276462,1.939519,2.763428,2.257679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.023664,2.989005,3.029789,3.329601,3.339253,3.130334,2.980912,2.930949,3.048442,3.107210,...,3.147676,3.357935,3.229682,3.155640,3.380392,3.342620,3.335057,3.254064,3.283979,3.160469
996.7096,2.830589,2.987219,1.556303,3.187803,3.226342,2.806180,2.602060,2.850033,2.722634,2.557507,...,3.347525,3.427324,3.448242,3.347720,3.351603,3.573220,3.501059,3.299071,3.452400,3.578066
997.5542,3.010724,2.758155,2.998695,2.867467,3.239800,3.044148,2.690196,3.158362,2.958564,2.992111,...,3.274850,3.565021,3.175222,3.422590,2.975432,3.433610,3.309417,3.374932,3.373464,3.475090


time: 30 ms (started: 2023-05-24 12:08:26 -05:00)


In [12]:
# Transpose

list_raw_t = []
for k in range(len(list_raw)):
  list_raw_t.append(transpose(list_raw[k]))

print(len(list_raw_t))
list_raw_t[0]

5


ionMz,59.0049,59.0137,59.0291,59.0370,59.0453,60.0171,60.9871,61.9884,71.0139,74.0249,...,992.8229,993.5530,994.9357,995.4648,995.9381,996.5509,996.7096,997.5542,997.7131,998.4845
0,2.515874,5.101637,3.683497,2.841985,0.000000,3.652826,3.473925,3.593618,3.508664,3.948706,...,2.599883,3.232488,3.585348,3.215373,2.984527,3.431203,2.877371,3.253822,3.077731,3.293584
1,2.453318,5.073656,3.647872,2.990783,2.475671,3.371253,3.427161,3.454387,3.409257,3.879612,...,0.000000,3.341039,3.518382,3.269513,3.174641,3.400538,3.203305,3.159868,2.881955,3.297104
2,2.832509,5.191384,3.772762,3.060320,2.632457,3.545925,2.926342,3.722140,3.668665,4.087249,...,2.800029,3.462697,3.698362,3.406881,3.173478,3.375481,3.030600,3.400538,2.924796,3.431525
3,2.687529,5.053144,3.684486,3.089198,2.597695,3.510947,3.204934,3.238046,3.497344,4.002857,...,2.893207,3.051538,3.703463,3.301247,2.998259,3.477266,3.000000,3.302547,2.472756,3.358125
4,2.515874,5.127630,3.743196,3.143951,2.862131,3.423246,2.837588,3.599446,3.591732,4.017242,...,3.497897,3.501880,3.744919,3.492341,3.175222,3.579669,3.068557,3.332842,2.747412,3.388279
5,1.973128,5.014327,3.626853,2.880242,2.468347,3.492062,3.434569,3.419460,3.491922,3.848189,...,2.737987,3.383636,3.551938,3.198657,3.261025,3.106191,3.232996,3.306639,2.990783,3.516800
6,2.873902,5.236834,3.847819,3.228657,2.568202,3.619928,3.212986,3.699404,3.598024,4.199974,...,2.915400,3.301247,3.623146,3.493458,3.496099,3.448397,2.970812,3.492201,3.084576,3.281033
7,2.670246,5.009944,3.593618,2.829947,2.553883,3.243782,3.219060,3.370698,3.365301,3.942157,...,2.457882,3.245266,3.542452,3.279211,2.925312,3.398461,3.075547,3.340841,2.878522,3.415140
8,2.795185,5.198135,3.759290,2.930949,2.403121,3.660771,3.326950,3.474944,3.656769,4.111867,...,2.578639,3.410440,3.657438,3.337459,3.097604,3.493179,3.157457,3.473633,3.174641,3.483159
9,2.675778,5.278424,3.860038,3.070776,2.113943,3.669131,3.280123,3.662096,3.732796,4.257463,...,2.802089,3.490380,3.636488,3.336860,3.043362,3.498724,3.283075,3.412461,3.011570,3.466571


time: 50.2 ms (started: 2023-05-22 18:07:03 -05:00)


In [13]:
# Correlation matrix

list_raw_corr = []
for df in tqdm(list_raw_t):
  matrix = df.corr(method="pearson") # pearson, kendall, spearman
  list_raw_corr.append(matrix)

print(len(list_raw_corr))
list_raw_corr[0]

100%|██████████| 5/5 [00:13<00:00,  2.77s/it]

5


ionMz,59.0049,59.0137,59.0291,59.0370,59.0453,60.0171,60.9871,61.9884,71.0139,74.0249,...,992.8229,993.5530,994.9357,995.4648,995.9381,996.5509,996.7096,997.5542,997.7131,998.4845
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1.000000,-0.147754,0.001130,0.126366,0.076868,-0.100973,-0.176885,-0.046750,-0.109445,0.068588,...,0.105425,-0.101016,0.020805,-0.335219,0.084352,0.065065,-0.309121,0.093378,0.030263,-0.030469
59.0137,-0.147754,1.000000,0.954735,0.341700,0.084760,0.680555,0.146571,0.656992,0.703167,0.371095,...,0.006019,0.425935,0.396302,0.460336,0.121486,0.350258,0.466286,0.309471,0.274854,0.355578
59.0291,0.001130,0.954735,1.000000,0.466228,0.175751,0.670839,0.083177,0.620450,0.695555,0.351730,...,0.109289,0.319478,0.389690,0.337168,0.125763,0.285102,0.443819,0.304211,0.295922,0.371843
59.0370,0.126366,0.341700,0.466228,1.000000,0.586452,0.266837,0.159436,0.355616,0.092568,0.393984,...,0.359698,-0.144315,0.158979,0.040625,0.158807,0.159979,0.330668,0.056445,0.331212,-0.026070
59.0453,0.076868,0.084760,0.175751,0.586452,1.000000,-0.129943,0.101367,-0.068070,-0.140435,0.146074,...,0.304700,-0.214060,-0.147247,-0.219534,-0.202576,-0.140144,0.156895,0.074908,0.369766,-0.046468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,0.065065,0.350258,0.285102,0.159979,-0.140144,0.130372,-0.053074,0.358575,0.341468,0.240389,...,-0.012183,0.383112,0.414269,0.519701,-0.088555,1.000000,0.092254,0.366922,-0.181307,0.437661
996.7096,-0.309121,0.466286,0.443819,0.330668,0.156895,0.386688,0.147265,0.312664,0.390001,0.246583,...,-0.162888,0.190810,0.132548,0.133271,-0.059767,0.092254,1.000000,-0.169739,-0.028362,0.039550
997.5542,0.093378,0.309471,0.304211,0.056445,0.074908,-0.024674,0.186720,0.188008,0.227667,0.128172,...,0.351046,0.450332,0.242205,0.360525,0.183490,0.366922,-0.169739,1.000000,0.307248,0.470779


time: 13.8 s (started: 2023-05-22 18:07:03 -05:00)


In [ ]:
# Build graph

threshold = 0.5
i = 1

for matrix in tqdm(list_raw_corr):
  edges = build_graph_weight(matrix, threshold)

  # save
  weighted_edges = pd.DataFrame(edges, columns=["source", "target", "weight"])
  weighted_edges.to_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], i), index=False)
  i += 1
  print(len(edges))

print(i - 1)
weighted_edges

 50%|██████████████████████                      | 1/2 [02:18<02:18, 138.54s/it]

1629510


100%|████████████████████████████████████████████| 2/2 [04:54<00:00, 147.14s/it]

4417355
2


,source,target,weight
0,59.0049,338.1873,0.507709
1,59.0049,357.2834,0.516811
2,59.0049,453.9675,0.507832
3,59.0049,587.1823,0.521553
4,59.0049,613.3933,0.516046
...,...,...,...
4417350,996.5509,997.5542,0.618624
4417351,996.5509,998.4845,0.678071
4417352,996.7096,997.5542,0.566913
4417353,997.5542,997.7131,0.589712


time: 4min 54s (started: 2023-05-19 10:27:17 -05:00)


In [ ]:
# Create dataset - nodes/edge data
for item in tqdm(subgroups):
    weighted_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], item))
    G = nx.from_pandas_edgelist(weighted_edges, "source", "target", edge_attr="weight")
    mapping = dict(zip(list(G.nodes()), range(G.number_of_nodes())))
    G = nx.relabel_nodes(G, mapping)
    degree = dict(G.degree())

    print("Subgroup: {}".format(item))
    graph_detail(G)

    df_nodes = pd.DataFrame(degree.items(), columns=["idx", "degree"])
    df_nodes["ionMz"] = list(mapping.keys())
    df_nodes.to_csv("{}/output_preprocessing/graph_data/{}_nodes_data_{}.csv".format(dir, group[0], item), index=False)

    edges = list(G.edges())
    df_edges = pd.DataFrame(edges, columns=["source", "target"])
    df_edges["weight"] = [G.get_edge_data(edge[0], edge[1])["weight"] for edge in edges]
    df_edges.to_csv("{}/output_preprocessing/graph_data/{}_edges_data_{}.csv".format(dir, group[0], item), index=False)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Subgroup: 1
Num. nodes: 6235
Num. edges: 1629510



 50%|██████████████████████▌                      | 1/2 [00:09<00:09,  9.96s/it]

Subgroup: 2
Num. nodes: 6234
Num. edges: 4417355



100%|█████████████████████████████████████████████| 2/2 [00:36<00:00, 18.07s/it]

time: 36.1 s (started: 2023-05-19 10:32:11 -05:00)
